<a href="https://colab.research.google.com/github/vitthal-bhandari/Homophobia-Transphobia-Detection/blob/master/Baselines/Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
!pip install datasets
!pip install transformers
!pip install pyspellchecker
!pip install emoji --upgrade
!sudo apt-get install git-lfs

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 136 kB 55.8 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 212 kB 54.2 MB/s 
     |████████████████████████████████| 127 kB 44.8 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 144 kB 57.5 MB/s 
     |████████████████████████████████| 271 kB 44.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 5.2 MB/s 


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from spellchecker import SpellChecker
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import emoji

In [5]:
notebook_login()

# Read Data

## Read English data

In [8]:
eng_train_dataset = load_dataset("csv", data_files="eng_3_train.tsv", sep="\t")
eng_train = eng_train_dataset["train"]
eng_dev_dataset = load_dataset("csv", data_files="eng_3_dev.tsv", sep="\t")
eng_dev = eng_dev_dataset["train"]
eng_test_dataset = load_dataset("csv", data_files="eng_3_test.tsv", sep="\t")
eng_test = eng_test_dataset["train"]
print(len(eng_train), len(eng_dev), len(eng_test))

Using custom data configuration default-23af21051fba98eb
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-23af21051fba98eb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-bc3a833de2e4bbdd
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-bc3a833de2e4bbdd/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-8dd80d81889d45a2
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-8dd80d81889d45a2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

3164 792 999


In [9]:
eng_train = eng_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
eng_train = eng_train.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_train = eng_train.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
eng_dev = eng_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9'])
eng_dev = eng_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_dev = eng_dev.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
eng_test = eng_test.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_test = eng_test.rename_column(original_column_name= 'text                        ', new_column_name= 'text')
eng_test = eng_test.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
# remove existing NA rows in test set
eng_test = eng_test.filter(lambda example, indice: indice < 990, with_indices=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
eng_train, eng_dev, eng_test

(Dataset({
     features: ['label', 'text'],
     num_rows: 3164
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 792
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 990
 }))

**Converting Datasets to Datafrane**

In [14]:
eng_train.set_format(type="pandas")
eng_dev.set_format(type="pandas")
eng_test.set_format(type="pandas")

df_eng_train = eng_train[:]
df_eng_dev = eng_dev[:]
df_eng_test = eng_test[:]

df_eng_train

,label,text
0,1,"I support her, very smart ponnu"
1,0,priyadharshini kannan same gender attraction ...
2,1,Bro u name and phone number (or)mobile number ...
3,1,experience Thaks bro I love you so much bro
4,1,world is becoming bad day by day....
...,...,...
3159,1,That Akka is really good soul
3160,1,Good madam.if i get any oppertunity in future ...
3161,1,judgment is based on scientific facts
3162,1,Respect to allll🙏🙏🙏


## Read Tamil data

In [ ]:
tam_train_dataset = load_dataset("csv", data_files="tam_3_train.tsv", sep="\t")
tam_train = tam_train_dataset["train"]
tam_dev_dataset = load_dataset("csv", data_files="tam_3_dev.tsv", sep="\t")
tam_dev = tam_dev_dataset["train"]
tam_test_dataset = load_dataset("csv", data_files="tam_3_test.tsv", sep="\t")
tam_test = tam_test_dataset["train"]
print(len(tam_train), len(tam_dev), len(tam_test))

Using custom data configuration default-b4e0e564b2a0ec6e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b4e0e564b2a0ec6e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-c40118cab1cdabc7


...

Using custom data configuration default-f53558da69751118


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f53558da69751118/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

2662 666 833


In [ ]:
tam_train, tam_dev, tam_test

(Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'],
     num_rows: 2662
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
     num_rows: 666
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed:

In [ ]:
tam_train = tam_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'])
tam_train = tam_train.rename_column(original_column_name= 'category', new_column_name= 'label')
tam_train = tam_train.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tam_dev = tam_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17'])
tam_dev = tam_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
tam_dev = tam_dev.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tam_test = tam_test.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'])
tam_test = tam_test.rename_column(original_column_name= 'category', new_column_name= 'label')
tam_test = tam_test.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tam_train, tam_dev, tam_test

(Dataset({
     features: ['label', 'text'],
     num_rows: 2662
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 666
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 833
 }))

**Converting Datasets to Datafrane**

In [ ]:
tam_train.set_format(type="pandas")
tam_dev.set_format(type="pandas")
tam_test.set_format(type="pandas")

df_tam_train = tam_train[:]
df_tam_dev = tam_dev[:]
df_tam_test = tam_test[:]

df_tam_train

,label,text
0,2,அர்த்தநாரீஸ்வரர் ஆணுமானவர் பெண்ணுமானவர். இது ஒ...
1,1,section377 | ஓரினச்சேர்க்கை நல்லதா கெட்டதா மக்...
2,2,இருக்கிற நாலு பேர் சேர்ந்து அந்த பொறம்போக்கு ச...
3,1,Shanth sha உடலுறவு இனப்பெருக்கத்திற்கு மட்டும...
4,1,ஜீவநதி ப்ரியன் hi
...,...,...
2657,0,நீங்கள் என்னதான் சட்டம் கொண்டுவந்தாலும் இது தவ...
2658,1,Ayy Ar இதய துடிப்பில் உனது நினைவலைகள். ஊன் உற...
2659,1,இவர்கள் இவ்வளவு பண்ணிவிட்டு எங்களை சமுதாயம் மத...
2660,1,ப்ரோ அவங்களுக்கு.திருநங்கைகள் தான் இவங்க.கட்டப...


## Read Code-Mixed data

In [ ]:
eng_tam_train_dataset = load_dataset("csv", data_files="eng_tam_3_train.tsv", sep="\t")
eng_tam_train = eng_tam_train_dataset["train"]
eng_tam_dev_dataset = load_dataset("csv", data_files="eng_tam_3_dev.tsv", sep="\t")
eng_tam_dev = eng_tam_dev_dataset["train"]
eng_tam_test_dataset = load_dataset("csv", data_files="eng_tam_3_test.tsv", sep="\t")
eng_tam_test = eng_tam_test_dataset["train"]
print(len(eng_tam_train), len(eng_tam_dev), len(eng_tam_test))

Using custom data configuration default-89805c3854dae1e4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-89805c3854dae1e4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-4e896a151151ec34


...

Using custom data configuration default-888a754b16e49ebb


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-888a754b16e49ebb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

3861 966 1207


In [ ]:
eng_tam_train, eng_tam_dev, eng_tam_test

(Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'],
     num_rows: 3861
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'],
     num_rows: 966
 }), Dataset({
     features: ['category', 'text', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed:

In [ ]:
eng_tam_train = eng_tam_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'
                                       , 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'])
eng_tam_train = eng_tam_train.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_tam_train = eng_tam_train.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
eng_tam_dev = eng_tam_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'])
eng_tam_dev = eng_tam_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_tam_dev = eng_tam_dev.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
eng_tam_test = eng_tam_test.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'
                                       , 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'])
eng_tam_test = eng_tam_test.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_tam_test = eng_tam_test.class_encode_column(column="label")

Casting to class labels:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
eng_tam_train, eng_tam_dev, eng_tam_test

(Dataset({
     features: ['label', 'text'],
     num_rows: 3861
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 966
 }), Dataset({
     features: ['label', 'text'],
     num_rows: 1207
 }))

**Converting Datasets to Datafrane**

In [ ]:
eng_tam_train.set_format(type="pandas")
eng_tam_dev.set_format(type="pandas")
eng_tam_test.set_format(type="pandas")

df_eng_tam_train = eng_tam_train[:]
df_eng_tam_dev = eng_tam_dev[:]
df_eng_tam_test = eng_tam_test[:]

df_eng_tam_train

,label,text
0,0,@Kadhayalla Nijam thaa gay Punda Apo ne confi...
1,1,love you too Nanba
2,1,Semma super 👍👍👍👍
3,1,Ava ka elom kadavuluka
4,1,Bro gay persons kalyanam panninaa kozhanda pet...
...,...,...
3856,1,Antha akka romba caring
3857,1,@Dan_Lei Nii Pooi Vidurayaa Unakku Yaarum v...
3858,1,Frnds neenga antha yedathula a irrunthaalum am...
3859,1,Looking pretty 🤩


## Drop empty rows

In [15]:
df_eng_train = df_eng_train.dropna()
df_eng_dev = df_eng_dev.dropna()
df_eng_test = df_eng_test.dropna()

df_eng_train = df_eng_train.reset_index(drop=True)
df_eng_dev = df_eng_dev.reset_index(drop=True)
df_eng_test = df_eng_test.reset_index(drop=True)

In [ ]:
df_tam_train = df_tam_train.dropna()
df_tam_dev = df_tam_dev.dropna()
df_tam_test = df_tam_test.dropna()

df_tam_train = df_tam_train.reset_index(drop=True)
df_tam_dev = df_tam_dev.reset_index(drop=True)
df_tam_test = df_tam_test.reset_index(drop=True)

In [ ]:
df_eng_tam_train = df_eng_tam_train.dropna()
df_eng_tam_dev = df_eng_tam_dev.dropna()
df_eng_tam_test = df_eng_tam_test.dropna()

df_eng_tam_train = df_eng_tam_train.reset_index(drop=True)
df_eng_tam_dev = df_eng_tam_dev.reset_index(drop=True)
df_eng_tam_test = df_eng_tam_test.reset_index(drop=True)

## Remove emojis

In [16]:
df_eng_train['text'] = df_eng_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_dev['text'] = df_eng_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_test['text'] = df_eng_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
df_tam_train['text'] = df_tam_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_tam_dev['text'] = df_tam_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_tam_test['text'] = df_tam_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
df_eng_tam_train['text'] = df_eng_tam_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_tam_dev['text'] = df_eng_tam_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_tam_test['text'] = df_eng_tam_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Remove unnecessary puncutation

In [17]:
#keras tokenizer

train_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
train_text_tok.fit_on_texts(df_eng_train['text'])

dev_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
dev_text_tok.fit_on_texts(df_eng_dev['text'])

test_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
test_text_tok.fit_on_texts(df_eng_test['text'])

In [18]:
train_tok = train_text_tok.texts_to_sequences(df_eng_train['text'])

df_eng_train['text'] = df_eng_train.apply(lambda row: train_text_tok.sequences_to_texts([train_tok[row.name]])[0], axis=1)

In [19]:
dev_tok = dev_text_tok.texts_to_sequences(df_eng_dev['text'])

df_eng_dev['text'] = df_eng_dev.apply(lambda row: dev_text_tok.sequences_to_texts([dev_tok[row.name]])[0], axis=1)

In [20]:
test_tok = test_text_tok.texts_to_sequences(df_eng_test['text'])

df_eng_test['text'] = df_eng_test.apply(lambda row: test_text_tok.sequences_to_texts([test_tok[row.name]])[0], axis=1)

## Run spell check

In [21]:
spell = SpellChecker()

spell.word_frequency.load_words(['lgbt', 'lgbtq', 'lgbtqia+', 'gay', 'lesbian', 'transgender', 'trans'])

In [22]:
def spell_check(row):
  correct_sent = []
  misspelled = spell.unknown(row.split())
  for word in row.split():
    if word in misspelled:
      correct_sent.append(spell.correction(word))
    else:
      correct_sent.append(word)

  return " ".join(correct_sent)

In [ ]:
df_eng_train['text'] = df_eng_train['text'].apply(spell_check)
df_eng_dev['text'] = df_eng_dev['text'].apply(spell_check)
df_eng_test['text'] = df_eng_test['text'].apply(spell_check)

**Review training dataframe once again**

In [ ]:
df_eng_train

,label,text
0,1,i support her very smart ponzu
1,0,priyadharshini kanan same gender attraction wi...
2,1,bro u name and phone number or mobile number p...
3,1,experience thanks bro i love you so much bro
4,1,world is becoming bad day by day
...,...,...
3157,1,that akka is really good soul
3158,1,good madam if i get any opportunity in future ...
3159,1,judgment is based on scientific facts
3160,1,respect to alll folded hands folded hands fold...


## Reset Dataset format

In [24]:
eng_train.reset_format()
eng_dev.reset_format()
eng_test.reset_format()

In [ ]:
tam_train.reset_format()
tam_dev.reset_format()
tam_test.reset_format()

In [ ]:
eng_tam_train.reset_format()
eng_tam_dev.reset_format()
eng_tam_test.reset_format()

# EDA

**Saving dataset in a csv format classwise separately**

In [ ]:
df_homophobic=df_eng_train[df_eng_train.label == 0]
df_homophobic.head()

,label,text
1,0,priyadharshini kanan same gender attraction wi...
22,0,really excellent movie i feel that no one are ...
31,0,madam read bible
94,0,it's due to abnormalities present sexual morph...
96,0,i think u too gay thats y u telling with the f...


In [ ]:
df_transphobic=df_eng_train[df_eng_train.label == 2]
df_transphobic.head()

,label,text
401,2,sex the 9
1090,2,magalakshmi mukunthan ella transgalayum konnud...
1340,2,today also one transgender scolded me for not ...
1418,2,rajeswari rajagopalan sister last week for not...
2330,2,hey seriously i thought she was a transgender


**Applying EDA to homophobia and transphobia classes**

In [ ]:
import nltk

nltk.download('wordnet')
!git clone https://github.com/jasonwei20/eda_nlp.git
%cd eda_nlp/code

!ls

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Cloning into 'eda_nlp'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 396 (delta 6), reused 10 (delta 4), pack-reused 379
Receiving objects: 100% (396/396), 20.42 MiB | 18.77 MiB/s, done.
Resolving deltas: 100% (189/189), done.
/content/eda_nlp/code
augment.py  eda.py  __pycache__


In [ ]:
df_homophobic.to_csv('homophobic.txt', sep="\t", header=False, index=False)
df_transphobic.to_csv('transphobic.txt', sep="\t", header=False, index=False)

In [ ]:
! python augment.py --input=transphobic.txt --num_aug=32

generated augmented sentences with eda for transphobic.txt to eda_transphobic.txt with num_aug=32


In [ ]:
! python augment.py --input=homophobic.txt --num_aug=16

generated augmented sentences with eda for homophobic.txt to eda_homophobic.txt with num_aug=16


**Combine augmented data with existing data**

In [ ]:
df_homophobic_aug=pd.read_csv('eda_homophobic.txt', sep="\t", names=["label", "text"])
df_transphobic_aug=pd.read_csv('eda_transphobic.txt', sep="\t", names=["label", "text"])

In [ ]:
df_homophobic_aug

,label,text
0,0,priyadharshini kanan same gender attraction is...
1,0,priyadharshini kanan same relevant attraction ...
2,0,with kanan same gender attraction with sex fir...
3,0,chum priyadharshini kanan same gender attracti...
4,0,priyadharshini kanan same gender attraction wi...
...,...,...
2664,0,importance savanna it seems that some people l...
2665,0,anu savannah it seems that some people likely ...
2666,0,anu savanna it likely that some people seems t...
2667,0,anu savanna it wrong that some people likely t...


In [ ]:
df_ally=df_eng_train[df_eng_train.label == 1]
df_ally.describe()

,label
count,2999.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
df_ally_shuffled=df_ally.sample(frac=1).reset_index(drop=True)
df_ally_shuffled_subset=df_ally_shuffled.iloc[:1500]

In [ ]:
df_ally_shuffled_subset

,label,text
0,1,sir please tell about kind of dreams and its i...
1,1,give your number
2,1,i understand your feelings
3,1,really is she transgender she is so cute havin...
4,1,don't worry everything will be solved soon
...,...,...
1495,1,no movie so sweet thne director sir
1496,1,such a beauty full supporting friends every ga...
1497,1,the one of the legend and good character only ...
1498,1,very sad but be safe and strong


In [ ]:
frames = [df_ally_shuffled_subset, df_homophobic, df_transphobic, df_homophobic_aug, df_transphobic_aug]
df_aug = pd.concat(frames)

In [ ]:
# re shuffling dataframe

df_aug_shuffled = df_aug.sample(frac=1).reset_index(drop=True)
df_aug_shuffled

,label,text
0,1,awesome real 1 gud try slightly smiling face
1,0,am i against to it
2,0,weird then why majority of gay couples has aid...
3,1,it's their own choice why should we brother if...
4,0,really excellent movie i feel that no one are ...
...,...,...
4525,0,need freedom but it doesnt means we more do an...
4526,0,dont give freedom for this type people wast ve...
4527,0,really excellent movie i feel that no one are ...
4528,1,latif kamal


In [ ]:
df_aug_shuffled['label'].value_counts()

0    2826
1    1500
2     204
Name: label, dtype: int64

# Saving data

**Save augmented english data**

In [ ]:
df_aug_shuffled.to_csv('eng_augmented_data.csv', index=False)

In [ ]:
df_eng_dev.to_csv('eng_dev.csv', index=False)

In [ ]:
df_eng_test.to_csv('eng_test.csv', index=False)

**Save english data without augmentation**

In [25]:
df_eng_train.to_csv('eng_train.csv', index=False)

**Save tamil data**

In [ ]:
df_tam_train.to_csv('tam_augmented_data.csv', index=False)

In [ ]:
df_tam_dev.to_csv('tam_dev.csv', index=False)

In [ ]:
df_tam_test.to_csv('tam_test.csv', index=False)

**Save code-mixed data**

In [ ]:
df_eng_tam_train.to_csv('eng_tam_augmented_data.csv', index=False)

In [ ]:
df_eng_tam_dev.to_csv('eng_tam_dev.csv', index=False)

In [ ]:
df_eng_tam_test.to_csv('eng_tam_test.csv', index=False)